In [1]:
from transformers import GPTJForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os


os.environ['HF_HOME']

/home/marat/miniconda/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/mnt/storage/HuggingFaceCache'

In [6]:
torch.backends.cudnn.version()

8401

In [7]:
torch.backends.cudnn.enabled == True

True

In [2]:
# import sys
# !{sys.executable} -m pip install accelerate

In [8]:
%%time
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float32, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

CPU times: user 5.08 s, sys: 14.7 s, total: 19.8 s
Wall time: 34.8 s


In [12]:
%%time
context = """In a shocking finding, scientists discovered a herd of unicorns living in a remote, 
            previously unexplored valley, in the Andes Mountains. Even more surprising to the 
            researchers was the fact that the unicorns spoke perfect English."""

input_ids = tokenizer(context, return_tensors="pt").input_ids
gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200, pad_token_id=tokenizer.eos_token_id)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

In a shocking finding, scientists discovered a herd of unicorns living in a remote, 
            previously unexplored valley, in the Andes Mountains. Even more surprising to the 
            researchers was the fact that the unicorns spoke perfect English. They said they 
            had come in response to an invitation sent to them by the National Geographic 
            Society. 
         </p>
         <p>
            The unicorns in question were not wild animals but were actually a group of unicorns 
            that were kept as pets by the Inca people. They were only wild for the
CPU times: user 14min 40s, sys: 6.38 s, total: 14min 46s
Wall time: 1min 50s


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/GPT-JT-6B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/GPT-JT-6B-v1")

In [3]:
model.to('cuda:0')

RuntimeError: CUDA out of memory. Tried to allocate 788.00 MiB (GPU 0; 23.69 GiB total capacity; 21.88 GiB already allocated; 568.19 MiB free; 21.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
%%time
context = """How many months does it take for two women to give birth to five babies with no twins?"""

input_ids = tokenizer(context, return_tensors="pt").input_ids
gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=100, pad_token_id=tokenizer.eos_token_id)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

NotImplementedError: Cannot copy out of meta tensor; no data!

In [3]:
model = AutoModelForCausalLM.from_pretrained('togethercomputer/GPT-JT-6B-v1', device_map='auto', load_in_8bit=True, int8_threshold=6.0)

TypeError: GPTJForCausalLM.__init__() got an unexpected keyword argument 'load_in_8bit'

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import dispatch_model, infer_auto_device_map
from accelerate.utils import get_balanced_memory

# Load model to CPU
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/GPT-JT-6B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/GPT-JT-6B-v1", device_map='auto')

max_memory = get_balanced_memory(
    model,
    max_memory=None,
    no_split_module_classes=["GPTJBlock"],
    dtype='float16',
    low_zero=False,
)

device_map = infer_auto_device_map(
    model,
    max_memory=max_memory,
    no_split_module_classes=["GPTJBlock"], 
    dtype='float16'
)

model = dispatch_model(model, device_map=device_map)

/home/marat/miniconda/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from transformers import pipeline
pipe = pipeline(model='togethercomputer/GPT-JT-6B-v1', model_kwargs= {"device_map": "auto", "load_in_8bit": True})

/home/marat/miniconda/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 5.37k/5.37k [00:00<00:00, 3.11MB/s]


TypeError: GPTJForCausalLM.__init__() got an unexpected keyword argument 'load_in_8bit'

In [ ]:
%%time
pipe('''"In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.''')